## Description:

Gets and stores data for nodes (representing street segments)

In [ ]:
import osmnx as ox
import folium
from geopy.distance import geodesic
from shapely.geometry import Polygon


def filter_edges_by_grid(edges, grid_square):
    print("filtering edges")
    west, south, east, north = grid_square
    grid_polygon = Polygon([(west, south), (west, north), (east, north), (east, south)])
    
    # Use the GeoDataFrame's spatial methods to check if edge geometries intersect with the grid square
    filtered_edges = edges[edges.geometry.apply(lambda geom: geom.intersects(grid_polygon))]
    return filtered_edges


# Function to interpolate points every 10 feet
def interpolate_points(start, end, step_ft):
    points = [start]
    total_distance = geodesic(start, end).feet
    step_ratio = step_ft / total_distance
    while step_ratio < 1.0:
        interp_lat = start[0] + (end[0] - start[0]) * step_ratio
        interp_lon = start[1] + (end[1] - start[1]) * step_ratio
        points.append((interp_lat, interp_lon))
        step_ratio += step_ft / total_distance
    points.append(end)  # Ensure the end point is included
    return points

def create_map_for_edges(edges):
    global grid_counter

    FEET_SPACING
    grid_counter += 1
    print("Creating map for Edges")
    point_counter = 0

    # Initialize a Folium map
    m = folium.Map(location=center_point, zoom_start=16)
    
    # Add edges to the map
    for _, row in edges.iterrows():
        coords = [(y, x) for x, y in row['geometry'].coords]
        
        # Interpolate points between each pair of coordinates
        for i in range(len(coords) - 1):
            start = coords[i]
            end = coords[i + 1]
            interpolated_points = interpolate_points(start, end, step_ft=FEET_SPACING)
            
            # Plot each interpolated point
            for point in interpolated_points:
                point_counter += 1
                # folium.Marker(location=point, popup=f"({location[0]}, {location[1]})").add_to(mymap)
                folium.CircleMarker(
                    location=point,
                    radius=2,
                    color='red',
                    fill=True,
                    fill_opacity=0.8,
                    popup=f"({point[0]}, {point[1]})"
                ).add_to(m)
    
    # Save the map
    m.save(f"data/dc_nodes/dc_{grid_counter}.html")
    print(f"Map created with {point_counter} points.")


def main():

    print("Starting")

    graph = ox.graph_from_place("Washington, DC, USA", network_type='drive')

    print("getting nodes and edges")
    nodes, edges = ox.graph_to_gdfs(graph)

    # prep for batching (since these is way to large of a dataset)
    bbox = edges.total_bounds  # [west, south, east, north]
    print("got bounds")


    # Divide the bounding box into 20 grids (4x5)
    num_rows = 6
    num_cols = 7
    grid_width = (bbox[2] - bbox[0]) / num_cols  # (east - west) / cols
    grid_height = (bbox[3] - bbox[1]) / num_rows  # (north - south) / rows

    grid_squares = []
    print("creating square")
    for row in range(num_rows):
        for col in range(num_cols):
            west = bbox[0] + col * grid_width
            east = west + grid_width
            south = bbox[1] + row * grid_height
            north = south + grid_height
            grid_squares.append((west, south, east, north))

    print(f"grid squares: {len(grid_squares)}")
    for idx, grid_square in enumerate(grid_squares):
        filtered_edges = filter_edges_by_grid(edges, grid_square)
        create_map_for_edges(filtered_edges)

grid_counter = 0
FEET_SPACING = 100
center_point = (38.90251234580012, -77.04350506490759)


main()


In [103]:
import osmnx as ox
point = (38.90373,-77.0488552)
# graph = ox.graph_from_place("Washington, DC, USA", network_type='drive')
graph = ox.graph_from_point(point,1000, network_type='drive')

nodes, edges = ox.graph_to_gdfs(graph)





In [104]:
import folium

# Create a folium map centered around the average latitude and longitude of the nodes
center_lat = nodes.geometry.y.mean()
center_lon = nodes.geometry.x.mean()
dc_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add a marker for each node
for node_id, node in nodes.iterrows():
    print(node)
    lat = node.geometry.y
    lon = node.geometry.x
    # folium.Marker(location=[lat, lon], popup=f"Node ID: {node_id}").add_to(dc_map)
    folium.CircleMarker(
                    location=[lat, lon],
                    radius=2,
                    color='red',
                    fill=True,
                    fill_opacity=0.8,
                    popup=f"({point[0]}, {point[1]})"
                ).add_to(dc_map)


# Add edges to the map with different colors for one-way and two-way edges
for _, edge in edges.iterrows():
    # Extract edge geometry
    if edge.geometry.geom_type == 'LineString':             
        coords = [(lat, lon) for lon, lat in edge.geometry.coords]
        color = 'red' if edge.get('oneway', False) else 'blue'
        folium.PolyLine(coords, color=color, weight=2.5, opacity=0.8).add_to(dc_map)
    elif edge.geometry.geom_type == 'MultiLineString':
        for line in edge.geometry:
            coords = [(lat, lon) for lon, lat in line.coords]
            color = 'red' if edge.get('oneway', False) else 'blue'
            folium.PolyLine(coords, color=color, weight=2.5, opacity=0.8).add_to(dc_map)

# Save or display the map
dc_map.save("data/test_washington_dc_nodes_map.html")
print("Saved")

# print(nodes)
# print(edges)

y                                    38.909644
x                                    -77.04772
street_count                                 3
highway                                    NaN
junction                                   NaN
geometry        POINT (-77.0477199 38.9096436)
Name: 49716899, dtype: object
y                                    38.908571
x                                   -77.048797
street_count                                 4
highway                                    NaN
junction                                   NaN
geometry        POINT (-77.0487972 38.9085711)
Name: 49717370, dtype: object
y                                    38.902397
x                                   -77.046651
street_count                                 4
highway                        traffic_signals
junction                                   NaN
geometry        POINT (-77.0466515 38.9023974)
Name: 49717477, dtype: object
y                                    38.905245
x                

In [91]:
from folium.plugins import PolyLineTextPath
from folium import Icon


def draw_segments_on_map(segments):
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    for segment_id, segment in segments.items():
        lat = segment['lat']
        long = segment['long']
        folium.CircleMarker(
            location=[lat, long],
            radius=2,
            color='red',
            fill=True,
            fill_opacity=0.8,
            popup=f" {segment_id} => ({segment['segment_links']})"
        ).add_to(m)

        # Draw lines to each segment link
        for linked_segment_id in segment['segment_links']:
            linked_segment = segments.get(linked_segment_id)
            if linked_segment:
                linked_lat = linked_segment['lat']
                linked_long = linked_segment['long']
                
                # Create a PolyLine (line) between the current segment and the linked segment
                folium.PolyLine(
                    locations=[(lat, long), (linked_lat, linked_long)],
                    color='blue',
                    weight=2,
                    opacity=0.5
                ).add_to(m)

    m.save("data/dc_segments_map.html")

In [105]:
import math
import pandas as pd
from geopy.distance import geodesic


def write_as_csv(filepath, dict):
    df = pd.DataFrame.from_dict(dict, orient='index')
    df.to_csv(filepath)

def segment_key(lat, long):
    return f"{lat}_{long}"


def calculate_heading(lat1, lon1, lat2, lon2):
    """
    Calculate the heading (bearing) from one point to another.
    Returns the heading in degrees.
    """
    d_lon = math.radians(lon2 - lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)

    x = math.sin(d_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))
    initial_heading = math.atan2(x, y)
    # Convert to degrees and normalize to 0-360
    heading = (math.degrees(initial_heading) + 360) % 360
    return heading

def add_all_nodes_to_segments(nodes, segments):
    # loop through all nodes and add them as segments
    for node_id, node in nodes.iterrows():
        lat = node.geometry.y
        long = node.geometry.x

        # print(node_id)
        # print(f"Node:{segment_key(lat,long)}")
        segment_headings = []
        segment_links = []

        successors = list(graph.successors(node_id))
        # print(successors)

        for successor_id in successors:
            successor_node = nodes.loc[successor_id]  
            successor_lat = successor_node.geometry.y
            successor_long = successor_node.geometry.x
            # print(f"  Successor Node ID: {successor_id}, Latitude: {successor_lat}, Longitude: {successor_long}")
            
            # # add segment link
            segment_links.append(segment_key(successor_lat, successor_long))

            # # calculate segment heading
            heading = calculate_heading(lat, long, successor_lat, successor_long)
            segment_headings.append(heading)


        # TODO debug
        segment_headings = []
        segment_links = []

        segments[segment_key(lat,long)] = {
            "lat": lat,
            "long": long, 
            "headings": segment_headings, 
            "segment_links": segment_links}



def add_segment_to_segments(segments, lat, long, headings, segment_links):
    segments[segment_key(lat,long)] = {
        "lat": lat,
        "long": long, 
        "headings": headings, 
        "segment_links": segment_links}



# attempts to add segments from an edge
def create_segments_from_edge(edge, segments, closed_edges):
    


    print(f"    u: {edge[0]}, v: {edge[1]}, data: {edge[2]}")
    if edge not in closed_edges:
        # we have not processed this edge yet...
        u = edge[0]
        v = edge[1]
        data = edge[2]

        oneway = bool(data['oneway'])


         

        # first get the distance between the two nodes
        u_node = nodes.loc[u]
        v_node = nodes.loc[v]
        u_lat = u_node.geometry.y
        u_long = u_node.geometry.x
        v_lat = v_node.geometry.y
        v_long = v_node.geometry.x

        u_key = segment_key(u_lat, u_long)
        v_key = segment_key(v_lat, v_long)
        
        next_heading = calculate_heading(u_lat, u_long, v_lat, v_long)
        # for two way streets we need to add the reverse heading
        reverse_heading = calculate_heading(v_lat, v_long, u_lat, u_long)

        print(f"(u_lat, u_long): ({u_lat}, {u_long}), (v_lat, v_long): ({v_lat}, {v_long})")
        distance_in_feet = geodesic((u_lat, u_long), (v_lat, v_long)).feet

        print(f"    Distance in feet: {distance_in_feet}")

        FEET_SPACING = 200
        # now attempt to divide the distance into segments of x feet
        segments_to_add = distance_in_feet // FEET_SPACING
        print(f"    Segments to add: {segments_to_add}")

        if segments_to_add == 0:
            # cant add any intermediate segments, so we must connect the u and v nodes
            # no matter what, we need to link u to v
            segments[u_key]['segment_links'].append(v_key)
            if not oneway:
                # then v can also point to u
                segments[v_key]['segment_links'].append(u_key)
                
            # no segments to add, so we can end early
            return

            

        # we want have segments equally between the two nodes
        distance_in_feet / (segments_to_add + 1)

        print (f"    Distance between segments: {distance_in_feet / (segments_to_add + 1)}")

        segment_locations = []
        for i in range(1, int(segments_to_add) + 1):
            print(f"    Adding Segment {i}")
            # determine what lat/long to put the segment at
            fraction = i / (segments_to_add + 1)  # Fraction between 0 and 1
            new_lat = float(u_lat) + fraction * (v_lat - u_lat)
            new_long = float(u_long) + fraction * (v_long - u_long)

            # print(f"    Segment {i}: (lat, long) = ({new_lat}, {new_long})")
            segment_locations.append((new_lat, new_long))

        print(f"    Segment Locations: {segment_locations}")
        for i, location in enumerate(segment_locations):
            new_segment_lat = location[0]
            new_segment_long = location[1]
            segment_key_str = segment_key(new_segment_lat, new_segment_long)
            print(f"    Segment {i}: {location}")
            print(f"    {bool(data['oneway'])}")

            # default
            
            previous_segment = u_key
            next_segment = v_key
            if i != 0:
                # previous location segment...
                previous_segment = segment_key(segment_locations[i - 1][0], segment_locations[i - 1][1])
            else: 
                # we are the first segment, so we need to tell node 'u' to link to us
                segments[u_key]['segment_links'].append(segment_key_str)

            if i != len(segment_locations) - 1:
                # next location segment...
                next_segment = segment_key(segment_locations[i + 1][0], segment_locations[i + 1][1])

            segment_links = []
            segment_headings = []

            # since we are are going in the outbound direction of a node (u -> v) we can assume
            # that the next segment in segment_locations is segment we need to link to (no matter what)
            # if we are not oneway (two way) then we need additionally need to link to the previous segment
            
            segment_links.append(next_segment)
            segment_headings.append(next_heading)
            if not oneway:
                print("    NOT ONEWAY")
                print(f"    adding Previous Segment: {previous_segment}")
                segment_links.append(previous_segment)
                segment_headings.append(reverse_heading)

            # add segment to segments
           
            print(f"    Creating Segment: {segment_key_str}, links: {segment_links}")
            if segment_key_str not in segments:
                segments[segment_key_str] = {
                    "lat": new_segment_lat,
                    "long": new_segment_long, 
                    "headings": segment_headings, 
                    "segment_links": segment_links}
                

        # add edge to closed edges
        closed_edges.append(edge)



def add_edges_between_nodes_to_segments(nodes, segments):
    closed_edges = [] # list of edges that have already been processed

    # loop through all nodes and add segements between them
    for node_id, node in nodes.iterrows():
        print(f"For Node ID: {node_id}")
        lat = node.geometry.y
        long = node.geometry.x

        print(node_id)
        print(f"Node:{segment_key(lat,long)}")
        

        
        # print(successors)

        succesor_edges = graph.edges(node_id, data=True)
        # print(f"    {edges}")
        for edge in succesor_edges:
            create_segments_from_edge(edge, segments, closed_edges)
            

        # loop through all succesor edges and try to add segments within them





def grab_store_all_segments(nodes, edges):
    segments = {} #key = lat_long, value = lat, long, headings[], segment_links[]

    add_all_nodes_to_segments(nodes, segments)
    add_edges_between_nodes_to_segments(nodes, segments)

    
    print("PRINTING SEGMENTS:")
    print(segments)
    write_as_csv("data/dc_segments.csv", segments)
    draw_segments_on_map(segments)


grab_store_all_segments(nodes, edges)


For Node ID: 49716899
49716899
Node:38.9096436_-77.0477199
    u: 49716899, v: 739515930, data: {'osmid': [6051300, 6051301], 'highway': 'residential', 'name': 'Twining Court Northwest', 'oneway': False, 'reversed': [False, True], 'length': np.float64(82.75802668032613), 'geometry': <LINESTRING (-77.048 38.91, -77.048 38.91, -77.048 38.909, -77.047 38.909, -...>}
(u_lat, u_long): (38.9096436, -77.0477199), (v_lat, v_long): (38.9092216, -77.0472746)
    Distance in feet: 199.20182792453522
    Segments to add: 0.0
    u: 49716899, v: 49779907, data: {'osmid': 130889721, 'highway': 'secondary', 'lanes': '4', 'name': 'P Street Northwest', 'oneway': False, 'reversed': False, 'length': np.float64(93.28365205182558), 'geometry': <LINESTRING (-77.048 38.91, -77.048 38.91, -77.048 38.91, -77.049 38.91, -77...>}
(u_lat, u_long): (38.9096436, -77.0477199), (v_lat, v_long): (38.909641, -77.048798)
    Distance in feet: 306.7947460802145
    Segments to add: 1.0
    Distance between segments: 153.

[segmentid] = lat, long, headings[], segment_links[], 

segment_links -> which other segment ids can you get to from this one.
for example, on a one way street, you can only get to the next one, but on a two way, you can get to the previous and next one...

Note, some segment links may not have a respective google maps tile (and thats okay), we still want to keep track of links in case we want to calculate pathing

make each intersection its own segment